---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---

# NBJ 3 — Interprétation fine d'un modèle : Integrated Gradients (IG)
### Outil XAI : Captum (PyTorch)

Dans ce TP, nous allons :
- utiliser PyTorch pour charger un modèle de vision (ResNet18) ;
- découvrir la bibliothèque **Captum**, spécialisée dans l’explicabilité ;
- expliquer une prédiction avec **Integrated Gradients (IG)** ;
- comparer IG à : Saliency (TP1) + Grad-CAM (TP2) ;
- révéler un **biais médical classique** : la règle chirurgicale.

Objectif : comprendre comment obtenir une explication **fiable, stable et précise**.

In [ ]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from captum.attr import IntegratedGradients, Saliency, LayerGradCam, LayerAttribution

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model = model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [ ]:
# TODO : ajouter une image de mélanome avec règle chirurgicale
img_path = "melanome_regle.jpg"  # à adapter

img = Image.open(img_path).convert("RGB")
input_tensor = transform(img).unsqueeze(0).to(device)

plt.imshow(img)
plt.axis("off")
plt.title("Image originale (mélanome avec règle)")

### ❓ Questions 1–2
1. Que représente cette image ? Vois-tu un élément qui pourrait tromper un modèle ?  
2. Selon toi, quelles zones **devraient** être utilisées pour classifier une lésion cutanée ?

In [ ]:
with torch.no_grad():
    output = model(input_tensor)
probs = torch.nn.functional.softmax(output[0], dim=0)
top_prob, top_idx = torch.max(probs, dim=0)

float(top_prob.item()), int(top_idx.item())

### ❓ Question 3  
Le modèle a-t-il raison ?  
Penses-tu qu’il regarde les **bonnes zones** ?

In [ ]:
saliency = Saliency(model)
input_tensor.requires_grad = True

grads = saliency.attribute(input_tensor, target=int(top_idx))
grads = grads.squeeze().cpu().detach().numpy()
saliency_map = np.max(np.abs(grads), axis=0)

plt.imshow(saliency_map, cmap='inferno')
plt.axis("off")
plt.title("Saliency Map (Captum)")

### ❓ Questions 4–5  
4. Quelles zones ressortent ?  
5. Cette explication te semble-t-elle claire, ou trop bruitée ?

In [ ]:
ig = IntegratedGradients(model)
attributions_ig = ig.attribute(
    input_tensor,
    target=int(top_idx),
    baselines=input_tensor*0  # baseline noire
)

attr = attributions_ig.squeeze().cpu().detach().numpy()
ig_map = np.max(np.abs(attr), axis=0)

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.imshow(img)
plt.title("Image originale")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(ig_map, cmap='inferno')
plt.title("Integrated Gradients")
plt.axis("off")
plt.tight_layout()

### ❓ Questions 6–8  
6. IG semble-t-il plus stable/plus net que la saliency brute ?  
7. IG met-il en évidence la **lésion**… ou la **règle** ?  
8. Pourquoi serait-ce une erreur grave en médecine ?

In [ ]:
target_layer = model.layer4[1].conv2
gradcam = LayerGradCam(model, target_layer)

gradcam_attr = gradcam.attribute(input_tensor, target=int(top_idx))
cam = LayerAttribution.interpolate(gradcam_attr, (224, 224)).squeeze().cpu().detach().numpy()

In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,3,1)
plt.imshow(saliency_map, cmap='inferno')
plt.title("Saliency brute")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(ig_map, cmap='inferno')
plt.title("Integrated Gradients")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(cam, cmap="jet")
plt.title("Grad-CAM")
plt.axis("off")

plt.tight_layout()

### ❓ Questions 9–12  
9. Compare les trois cartes : lesquelles sont propres ? lesquelles sont bruitées ?  
10. Quelle méthode identifie le mieux les *contours* ?  
11. Quelle méthode montre plutôt les *zones globales* ?  
12. Laquelle te semble la **meilleure explication** pour ce cas précis ? Pourquoi ?

# 🧠 Synthèse — Le biais de la règle chirurgicale

- Beaucoup de modèles de dermatologie mal entraînés apprennent un **faux indice** :
  → la présence d’une règle (utilisée pour mesurer la lésion).

- Integrated Gradients a mis en évidence ce biais :
  → la règle reçoit plus d’importance que la lésion.

- Cela montre que :
  **Une IA peut avoir raison mais pour les mauvaises raisons.**

### ❓ Question finale  
Selon toi, comment pourrait-on **corriger** ce biais dans l’entraînement ?

---

### 📜 Licence d'utilisation

Ce document est protégé sous licence **Creative Commons BY-NC-ND 4.0 International**  
🔒 **Aucune modification ni réutilisation sans autorisation explicite de l'auteur.**

- 👤 Auteur : Christie Vassilian  
- 📥 Téléchargement autorisé uniquement à usage pédagogique personnel  
- 🚫 Réutilisation commerciale ou modification interdite  

[![Licence CC BY-NC-ND](https://licensebuttons.net/l/by-nc-nd/4.0/88x31.png)](https://creativecommons.org/licenses/by-nc-nd/4.0/)

---